In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import networkx as nx
import pandas as pd
import os 
import seaborn as sns


In [2]:
def E_lambda(V, i):
    N = V.shape[0]
    row_mat = np.tile(V[:,i],(N,1))
    E = (row_mat - row_mat.T)**2
    return E

def Sn(V, i):
    N = len(V)
    
    if i==(N-1):
        return E_lambda(V, N-1)
    else:
        return Sn(V, i+1) + E_lambda(V, i)
#def Sn(V, i):
#    N = len(V)
#    sn=np.zeros((N,N))
#    for k in range(N-1,i-1,-1):
#        sn+=E_lambda(V,k)
#    return sn
#x = Sn(V,4)    

def save_Sn(V, fname):
    N = len(V)
    savez_dict = dict()
    
    for k in range(N-1,-1,-1):
        savez_dict["S_%d"%(k+1)] = Sn(V,k) 

    np.savez("%s.npz"%fname, **savez_dict)

# finding clusters
def find_clusters(file):
    clusters = []
    lambda_indplus1 = []
    lambda_critical = []
    cumulative_cluster = []
    N = np.load(file)["S_1"].shape[0]
    
    for i in range(N,0,-1):
        
        clmat = np.load(file)["S_%d"%i]
        ind_list = np.argwhere(np.isclose(clmat, 2.0))
        cluster = list( np.array(list(  set([j for sublist in ind_list for j in sublist])  ))+1)
        
        #cluster = np.unique(np.argwhere(np.isclose(clmat, 2.0))[:,0])+1
        
        if len(cluster)>len(cumulative_cluster):
            
            cluster = list( set(cluster) - set(cumulative_cluster) )
            cumulative_cluster += cluster
            #cumulative_cluster = list( np.array(list(  set([j for sublist in ind_list for j in sublist])  ))+1)
            
            lambda_indplus1.append(i)
            clusters.append(sorted(cluster))
    
    return lambda_indplus1, clusters          


In [5]:
if not os.path.exists("./data/"): 
    os.makedirs("./data/") 

if not os.path.exists("./data/SmallWeighted/"): 
    os.makedirs("./data/SmallWeighted/") 

if not os.path.exists("./data/NCM/"): 
    os.makedirs("./data/NCM/") 


## Small weighted toy network

In [6]:
A = np.array([
                [0]+[0.1]*9, 
                 [0.1]+[0]+[0.1]*8, 
                 [0.1]*2+[0]+[0.1]*7, 
                 [0.1]*3+[0]+[0.529]*6,
                [0.1]*3+[0.529]+[0]+[0.529]*5,
                [0.1]*3+[0.529]*2+[0]+[0.529]*4,
                 [0.1]*3+[0.529]*3+[0]+[1.029]*3,
                 [0.1]*3+[0.529]*3+[1.029]+[0]+[1.029]*2,
                 [0.1]*3+[0.529]*3+[1.029]*2+[0]+[1.029],
                 [0.1]*3+[0.529]*3+[1.029]*3+[0]
                 ]) 
D = np.diag(np.sum(A, axis=0))
L = D-A

lamda, V = np.linalg.eigh(L)



In [7]:
save_Sn(V, "./data/SmallWeighted/sn_matrices_toy")

In [8]:
filename = "./data/SmallWeighted/sn_matrices_toy.npz"
lambda_indplus1, clusters = find_clusters(filename)
print(lambda_indplus1)
print(clusters)
lindp1_cluster_dict = dict(zip(lambda_indplus1, clusters))

[8, 5, 2]
[[7, 8, 9, 10], [4, 5, 6], [1, 2, 3]]


In [9]:
if not os.path.exists("./images/"): 
	os.makedirs("./images/") 

if not os.path.exists("./images/SmallWeighted"): 
	os.makedirs("./images/SmallWeighted") 


In [10]:
for l_indp1 in lambda_indplus1:
    plt.figure(figsize = (6,5))
    plt.rcParams.update({'font.size': 10})
    
    sn_mat = np.load(filename)["S_%d"%l_indp1]
    N = sn_mat.shape[0]
    
    sns.heatmap(sn_mat, cmap = 'rainbow', vmin = 0, vmax = 2, 
                    annot=True, fmt=".1f", square=True, 
                linewidths=.5, yticklabels=range(1,N+1), xticklabels=range(1,N+1))
    numbers_str = ', '.join(map(str, lindp1_cluster_dict[l_indp1]))
    plt.title("$\lambda_%d = %.2f$ \n cluster added = [%s]"%(l_indp1,lamda[l_indp1],numbers_str ), size=9)
    plt.savefig("./images/SmallWeighted/lambda_%d.pdf"%l_indp1, facecolor="white", bbox_inches="tight", dpi=600)
    plt.close()


In [11]:
# store results as dataframe
# 0.157 is critical coupling strength of Rossler y-y coupled parameter (0.2, 0.2, 9)
# which has MSF class II behaviour.
df = pd.DataFrame({"lambda_index":lambda_indplus1,
             "lambda":lamda[np.array(lambda_indplus1)-1],
                   "dc":0.157/np.array(lamda[np.array(lambda_indplus1)-1]),
             "cluster":clusters})
df.to_csv("./data/SmallWeighted/toy_clusters.csv", index=False)

## Experimental data brain

In [12]:
if not os.path.exists("./data/NCM"): 
	os.makedirs("./data/NCM") 

if not os.path.exists("./images/NCM"): 
	os.makedirs("./images/NCM") 


In [14]:
# Experimental data
A = np.loadtxt(open("./data/G.csv", "rb"), delimiter=",").astype("float")
D = np.diag(np.sum(A, axis=0))
L = D-A
lamda, V = np.linalg.eigh(L)


In [15]:
save_Sn(V, "./data/NCM/sn_matrices_ncm")

In [16]:
np.savez("./data/NCM/spectral_data.npz", {"eigenValues":lamda, "eigenVectors":V} )

In [17]:
filename = "./data/NCM/sn_matrices_ncm.npz"
lambda_indplus1, clusters = find_clusters(filename)
print(lambda_indplus1)
print(clusters)
lindp1_cluster_dict = dict(zip(lambda_indplus1, clusters))

[55, 50, 45, 41, 32, 28, 27, 25, 24, 23, 21, 20, 19, 18, 13, 11, 9, 8, 7, 6, 5, 4, 3, 2]
[[16, 41], [38, 52, 56], [3, 7], [2, 4, 28, 36], [5, 48], [21, 47], [11, 13, 27, 31, 32, 35, 40], [20, 34, 39, 54], [1], [45], [8, 9, 42, 46], [55], [53], [10, 51], [17], [29, 37, 59], [15, 24], [26], [6, 30], [58], [12, 14, 18, 22, 23, 57], [19], [25, 33, 49, 50], [43, 44]]


In [18]:
filename = "./data/NCM/sn_matrices_ncm.npz"
# sort matrix by cluster
ind_arr = np.array([x for xs in clusters[::-1] for x in xs])-1

for l_indp1 in lambda_indplus1:
    plt.figure(figsize = (13,9))
    plt.rcParams.update({'font.size': 8})
    
    sn_mat = np.load(filename)["S_%d"%l_indp1]
    N = sn_mat.shape[0]

    # sort matrix by ind array
    sn_mat = sn_mat[ind_arr,:][:,ind_arr]
    sns.heatmap(sn_mat, cmap = 'rainbow', vmin = 0, vmax = 2, 
                linewidths=.4, yticklabels=ind_arr+1, xticklabels=ind_arr+1)
    #annot=True, fmt=".1f", square=True,
    numbers_str = ', '.join(map(str, lindp1_cluster_dict[l_indp1]))
    plt.title("$\lambda_{%d} = %.2f$ \n cluster added = [%s]"%(l_indp1,lamda[l_indp1],numbers_str ), size=10)
    plt.savefig("./images/NCM/lambda_%d.pdf"%l_indp1, facecolor="white", bbox_inches="tight", dpi=600)
    plt.close()


In [31]:
# Predicted clusters
df = pd.DataFrame({"lambda_index":lambda_indplus1,
             "lambda":lamda[np.array(lambda_indplus1)-1],
             "cluster":clusters})
# reject all clusters whose length is = 1
df = df[df['cluster'].apply(lambda x: len(x) != 1)]
df.to_csv("./data/NCM/pred_clusters.csv", index=False)

In [22]:
# MSF class III
import ast
df = pd.DataFrame({"lambda_index":lambda_indplus1,
             "lambda":lamda[np.array(lambda_indplus1)-1],
             "dc1":0.167/np.array(lamda[np.array(lambda_indplus1)-1]),
             "dc2":23.2/np.array(lamda[np.array(lambda_indplus1)-1]),                       
             "cluster":clusters})
# reject all clusters whose length is = 1
df = df[df['cluster'].apply(lambda x: len(x) != 1)]
df.to_csv("./data/NCM/ncm_clusters_msf3.csv", index=False)


In [23]:
df

,lambda_index,lambda,dc1,dc2,cluster
0,55,46.000000,0.003630,0.504348,"[16, 41]"
1,50,41.000000,0.004073,0.565854,"[38, 52, 56]"
2,45,37.000000,0.004514,0.627027,"[3, 7]"
3,41,36.000000,0.004639,0.644444,"[2, 4, 28, 36]"
4,32,27.000000,0.006185,0.859259,"[5, 48]"
5,28,24.000000,0.006958,0.966667,"[21, 47]"
6,27,23.914214,0.006983,0.970134,"[11, 13, 27, 31, 32, 35, 40]"
7,25,17.941379,0.009308,1.293100,"[20, 34, 39, 54]"
10,21,10.962734,0.015233,2.116260,"[8, 9, 42, 46]"
13,18,7.000000,0.023857,3.314286,"[10, 51]"


In [24]:
print(df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.6f}".format,))

\begin{tabular}{rrrrl}
\toprule
lambda_index & lambda & dc1 & dc2 & cluster \\
\midrule
55 & 46.000000 & 0.003630 & 0.504348 & [16, 41] \\
50 & 41.000000 & 0.004073 & 0.565854 & [38, 52, 56] \\
45 & 37.000000 & 0.004514 & 0.627027 & [3, 7] \\
41 & 36.000000 & 0.004639 & 0.644444 & [2, 4, 28, 36] \\
32 & 27.000000 & 0.006185 & 0.859259 & [5, 48] \\
28 & 24.000000 & 0.006958 & 0.966667 & [21, 47] \\
27 & 23.914214 & 0.006983 & 0.970134 & [11, 13, 27, 31, 32, 35, 40] \\
25 & 17.941379 & 0.009308 & 1.293100 & [20, 34, 39, 54] \\
21 & 10.962734 & 0.015233 & 2.116260 & [8, 9, 42, 46] \\
18 & 7.000000 & 0.023857 & 3.314286 & [10, 51] \\
11 & 3.000000 & 0.055667 & 7.733333 & [29, 37, 59] \\
9 & 1.000000 & 0.167000 & 23.200000 & [15, 24] \\
7 & 0.972591 & 0.171706 & 23.853801 & [6, 30] \\
5 & 0.961943 & 0.173607 & 24.117849 & [12, 14, 18, 22, 23, 57] \\
3 & 0.807418 & 0.206832 & 28.733582 & [25, 33, 49, 50] \\
2 & 0.594452 & 0.280931 & 39.027519 & [43, 44] \\
\bottomrule
\end{tabular}



In [25]:
# MSF class II
df = pd.DataFrame({"lambda_index":lambda_indplus1,
             "lambda":lamda[np.array(lambda_indplus1)-1],
             "dc":0.157/np.array(lamda[np.array(lambda_indplus1)-1]),          
             "cluster":clusters})
# reject all clusters whose length is = 1
df = df[df['cluster'].apply(lambda x: len(x) != 1)]

df.to_csv("./data/NCM/ncm_clusters_msf2.csv", index=False)


In [26]:
df

,lambda_index,lambda,dc,cluster
0,55,46.000000,0.003413,"[16, 41]"
1,50,41.000000,0.003829,"[38, 52, 56]"
2,45,37.000000,0.004243,"[3, 7]"
3,41,36.000000,0.004361,"[2, 4, 28, 36]"
4,32,27.000000,0.005815,"[5, 48]"
5,28,24.000000,0.006542,"[21, 47]"
6,27,23.914214,0.006565,"[11, 13, 27, 31, 32, 35, 40]"
7,25,17.941379,0.008751,"[20, 34, 39, 54]"
10,21,10.962734,0.014321,"[8, 9, 42, 46]"
13,18,7.000000,0.022429,"[10, 51]"


In [27]:
print(df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.6f}".format,))

\begin{tabular}{rrrl}
\toprule
lambda_index & lambda & dc & cluster \\
\midrule
55 & 46.000000 & 0.003413 & [16, 41] \\
50 & 41.000000 & 0.003829 & [38, 52, 56] \\
45 & 37.000000 & 0.004243 & [3, 7] \\
41 & 36.000000 & 0.004361 & [2, 4, 28, 36] \\
32 & 27.000000 & 0.005815 & [5, 48] \\
28 & 24.000000 & 0.006542 & [21, 47] \\
27 & 23.914214 & 0.006565 & [11, 13, 27, 31, 32, 35, 40] \\
25 & 17.941379 & 0.008751 & [20, 34, 39, 54] \\
21 & 10.962734 & 0.014321 & [8, 9, 42, 46] \\
18 & 7.000000 & 0.022429 & [10, 51] \\
11 & 3.000000 & 0.052333 & [29, 37, 59] \\
9 & 1.000000 & 0.157000 & [15, 24] \\
7 & 0.972591 & 0.161424 & [6, 30] \\
5 & 0.961943 & 0.163211 & [12, 14, 18, 22, 23, 57] \\
3 & 0.807418 & 0.194447 & [25, 33, 49, 50] \\
2 & 0.594452 & 0.264109 & [43, 44] \\
\bottomrule
\end{tabular}



In [28]:
df_2 = pd.read_csv("./data/NCM/ncm_clusters_msf2.csv")
df_3 = pd.read_csv("./data/NCM/ncm_clusters_msf3.csv")

df = pd.DataFrame({"cluster no.":range(1,len(df_2)+1),
                  "lambda_index":df_2["lambda_index"],
                  "lambda":df_2["lambda"],
                  "dc":df_2["dc"],
                  "dc1":df_3["dc1"],
                  "dc2":df_3["dc2"],
                   "cluster":df_2["cluster"]
                  })

In [29]:
df

,cluster no.,lambda_index,lambda,dc,dc1,dc2,cluster
0,1,55,46.000000,0.003413,0.003630,0.504348,"[16, 41]"
1,2,50,41.000000,0.003829,0.004073,0.565854,"[38, 52, 56]"
2,3,45,37.000000,0.004243,0.004514,0.627027,"[3, 7]"
3,4,41,36.000000,0.004361,0.004639,0.644444,"[2, 4, 28, 36]"
4,5,32,27.000000,0.005815,0.006185,0.859259,"[5, 48]"
5,6,28,24.000000,0.006542,0.006958,0.966667,"[21, 47]"
6,7,27,23.914214,0.006565,0.006983,0.970134,"[11, 13, 27, 31, 32, 35, 40]"
7,8,25,17.941379,0.008751,0.009308,1.293100,"[20, 34, 39, 54]"
8,9,21,10.962734,0.014321,0.015233,2.116260,"[8, 9, 42, 46]"
9,10,18,7.000000,0.022429,0.023857,3.314286,"[10, 51]"


In [30]:
df.to_latex()

'\\begin{tabular}{lrrrrrrl}\n\\toprule\n & cluster no. & lambda_index & lambda & dc & dc1 & dc2 & cluster \\\\\n\\midrule\n0 & 1 & 55 & 46.000000 & 0.003413 & 0.003630 & 0.504348 & [16, 41] \\\\\n1 & 2 & 50 & 41.000000 & 0.003829 & 0.004073 & 0.565854 & [38, 52, 56] \\\\\n2 & 3 & 45 & 37.000000 & 0.004243 & 0.004514 & 0.627027 & [3, 7] \\\\\n3 & 4 & 41 & 36.000000 & 0.004361 & 0.004639 & 0.644444 & [2, 4, 28, 36] \\\\\n4 & 5 & 32 & 27.000000 & 0.005815 & 0.006185 & 0.859259 & [5, 48] \\\\\n5 & 6 & 28 & 24.000000 & 0.006542 & 0.006958 & 0.966667 & [21, 47] \\\\\n6 & 7 & 27 & 23.914214 & 0.006565 & 0.006983 & 0.970134 & [11, 13, 27, 31, 32, 35, 40] \\\\\n7 & 8 & 25 & 17.941379 & 0.008751 & 0.009308 & 1.293100 & [20, 34, 39, 54] \\\\\n8 & 9 & 21 & 10.962734 & 0.014321 & 0.015233 & 2.116260 & [8, 9, 42, 46] \\\\\n9 & 10 & 18 & 7.000000 & 0.022429 & 0.023857 & 3.314286 & [10, 51] \\\\\n10 & 11 & 11 & 3.000000 & 0.052333 & 0.055667 & 7.733333 & [29, 37, 59] \\\\\n11 & 12 & 9 & 1.000000 & 0.1